In [5]:
%%time
from pystac_client import Client
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.io import Reader
from rio_tiler.models import ImageData
import morecantile

tms = morecantile.tms.get("WebMercatorQuad")
x, y, z = tms.tile(-93,45,14)
bbox = list(tms.bounds(morecantile.Tile(x, y, z)))
print(bbox)

def reader(asset: str, *args, **kwargs) -> ImageData:
    with Reader(asset) as src:
        return src.tile(*args, **kwargs)


catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
results = catalog.search(
    limit=10,
    max_items=100,
    bbox=bbox,
    collections=["naip"],
    fields={"include":["assets.image.href"], "exclude":["links"]}
)
print(results.url_with_parameters())
items=results.items_as_dicts()

assets = (item['assets']['image']['href'] for item in items)

img, used = mosaic_reader(assets, reader, x, y, z, threads=4)
print(len(used))


[-93.01025390625037, 44.99588261816585, -92.98828125000037, 45.01141864227766]
https://planetarycomputer.microsoft.com/api/stac/v1/search?limit=10&bbox=-93.01025390625037%2C44.99588261816585%2C-92.98828125000037%2C45.01141864227766&collections=naip&fields=
4 4 4
4
CPU times: user 383 ms, sys: 7.44 ms, total: 391 ms
Wall time: 1.08 s
